<a href="https://colab.research.google.com/github/davidhuynhgit/BCC_LMS/blob/main/BCC_LMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
# Load acronym intergrated data
acronym = pd.read_csv("Library Checkouts — Heading acronym list — metadata.csv")
acronym.head()

In [ ]:
# Extract Branch data as a table
# Difference compared with the Library location table
# 1. They all go with "Library"
# 2. Mt should come with no period (.)
# 3. BrackenRidge is separate
branch = acronym.iloc[:, [0, 1]].dropna()
for index, row_data in branch.iterrows():
  branch_name = str(row_data.iloc[1])
  if "BrackenRidge" in branch_name:
    branch.iloc[index, 1] = branch_name.replace("BrackenRidge", "Bracken Ridge")
  if "Mt." in branch_name:
    branch.iloc[index, 1] = branch_name.replace("Mt.", "Mt")
  if "library" not in str(row_data.iloc[1]).lower():
    branch.iloc[index, 1] = str(row_data.iloc[1]) + " Library"
branch.to_csv('Branch.csv', index=False)

In [ ]:
# Extract Heading data as metadata
headings = acronym.iloc[:, [2, 3]].dropna()
headings
headings.to_csv('Headings.csv', index=False)

In [ ]:
# Extract ItemType data as a table
items = acronym.iloc[:, [4, 5]].dropna()
items
items.to_csv('ItemType.csv', index=False)

In [ ]:
# Load libraries' location data
location = pd.read_csv("libraries-information-location.csv")
location.head(2)

,Venue,Latitude,Longitude,Website,Address,Open,Parking,Phone,Summary,Facilities,Meeting rooms,geolocation
0,Annerley Library,-27.509341,153.033310,https://www.brisbane.qld.gov.au/libraries-venues-and-facilities/libraries/library-locations/annerley-library,"450 Ipswich Rd, Annerley",Monday: 10am-5pm\nTuesday: 10am-5pm\nWednesday: closed\nThursday: closed\nFriday: 10am-5pm\nSaturday: 9am-4pm\nSunday: closed\nClosed on public holidays,On street;\nOff street,07 3403 1735,"The Annerley Library features a variety of facilities and services, as well as plenty of activities and events. Limited wheelchair access, phone the library for more information.","Use the website link for collections, facilities, opening hours, parking, location, upcoming events for this library; Limited wheelchair access, phone the library for more information.",No,"-27.509341, 153.03331"
1,Ashgrove Library,-27.443876,152.987086,https://www.brisbane.qld.gov.au/libraries-venues-and-facilities/libraries/library-locations/ashgrove-library,"87 Amarina Ave, Ashgrove",Monday: 10am-6pm\nTuesday: 10am-8pm\nWednesday: 10am-6pm\nThursday: 10am-8pm\nFriday: 10am-6pm\nSaturday: 9am-4pm\nSunday: closed\nClosed on public holidays,On street;\nOff street,07 3407 1940,"The Ashgrove Library features a variety of facilities and services, as well as plenty of activities and events. It is wheelchair accessible and has community meeting rooms available for hire, including access to a kitchenette.","Use the website link for collections, facilities, opening hours, meeting rooms, parking, location, upcoming events and phone number information for this library; wheelchair accessible.",Yes;\nHearing loop;\nWheelchair accessibility;\nWheelchair accessible toilets;\nUse the website link for meeting room details,"-27.443876, 152.987086"


In [ ]:
# For SQL practice, there are several columns that are not important to our report. So just drop them as their insights are not significant.
# Notice that, the Opening hours greatly vary among libraries. Hence, it is reasonable to drop this as a redundant column.
# Create a list of essential columns
columns_to_keep = ['Venue', 'Address', 'Parking', 'Facilities', 'Meeting rooms']
# Keep only the essential columns and drop others
location = location[columns_to_keep]
location.head(2)

,Venue,Address,Parking,Facilities,Meeting rooms
0,Annerley Library,"450 Ipswich Rd, Annerley",On street;\nOff street,"Use the website link for collections, facilities, opening hours, parking, location, upcoming events for this library; Limited wheelchair access, phone the library for more information.",No
1,Ashgrove Library,"87 Amarina Ave, Ashgrove",On street;\nOff street,"Use the website link for collections, facilities, opening hours, meeting rooms, parking, location, upcoming events and phone number information for this library; wheelchair accessible.",Yes;\nHearing loop;\nWheelchair accessibility;\nWheelchair accessible toilets;\nUse the website link for meeting room details


In [ ]:
# Let's dive deeper into Open, Parking, Facilities, and Meeting rooms because they contain detailed information that need to be split and explored.
# Street Parking: to simplify, we can map as Street Parking, Shopping Centre and Not available
# Facilities: just keep the text after semi-colon
# Meeting rooms: only Yes or No are enough to figure out
# Define a function to map parking types
for index, row_data in location.iterrows():
  parking = str(row_data.iloc[2])
  if "street" in parking.lower():
      location.iloc[index, 2] = "Street parking"
  elif "Shopping Centre" in parking:
      location.iloc[index, 2] = "Shopping Centre"
  else:
      location.iloc[index, 2] = "Not available"

  facilities = str(row_data.iloc[3])
  location.iloc[index, 3] = facilities.split(";")[-1]

  meeting_room = str(row_data.iloc[4])
  if "no" in meeting_room.lower():
    location.iloc[index, 4] = "No"
  else:
    location.iloc[index, 4] = "Yes"

,Venue,Address,Parking,Facilities,Meeting rooms
0,Annerley Library,"450 Ipswich Rd, Annerley",Street parking,"Limited wheelchair access, phone the library for more information.",No
1,Ashgrove Library,"87 Amarina Ave, Ashgrove",Street parking,wheelchair accessible.,Yes
2,Banyo Library,"284 St Vincents Rd, Banyo",Street parking,wheelchair accessible.,Yes
3,Bracken Ridge Library,"Cnr Bracken and Barrett Street, Bracken Ridge",Street parking,wheelchair accessible.,Yes
4,Brisbane Square Library,"266 George St, Brisbane City",Not available,wheelchair accessible.,Yes


In [ ]:
# Looks great now, let's save this as a table
location.to_csv('Location.csv', index=False)

In [ ]:
# It's time to investigate the fact table of library checkout in 2024
# Load checkout data
checkout = pd.read_csv("library-checkouts-2024.csv")
checkout.head(5)

/tmp/ipython-input-2-1896678583.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  checkout = pd.read_csv("library-checkouts-2024.csv")


,Title,Author,Call Number,Item Id,Item Type,Status,Language,Age,Checkout Library,Date
0,The abolition of Britain : from Winston Churchill to Theresa May /,"Hitchens, Peter,",941.085 HIT,34000106965783,NONFICTION,CHECKEDOUT,NaN,ADULT,KEN,2024-12-09T23:57:56+10:00
1,The snakehead : an epic tale of the Chinatown underworld and the American dream /,"Keefe, Patrick Radden,",364.137 KEE,34000117610220,NONFICTION,CHECKEDOUT,NaN,ADULT,KEN,2024-12-09T23:57:49+10:00
2,What birdo is that? : a field guide to bird-people /,"Robin, Libby,",598.072 ROB,34000118196203,NONFICTION,CHECKEDOUT,NaN,ADULT,IPY,2024-12-09T23:55:46+10:00
3,Women making waves : trailblazing surfers in and out of the water /,"Einzig, Lara,",797.32 EIN,34000116161050,NONFICTION,CHECKEDOUT,NaN,ADULT,IPY,2024-12-09T23:55:40+10:00
4,The next big thing /,"Colley, James,",AD-PBK COL,34000119419448,AD-PBK,CHECKEDOUT,NaN,ADULT,IPY,2024-12-09T23:55:37+10:00


In [ ]:
# Interesting! At the first glance, the "/" at the end of items' title should be removed to improve readability.
checkout['Title'] = checkout['Title'].str.replace(r' /$', '', regex=True)

# According to the acronym list, the default language is English, equivalent to NaN. To make it clear and precise, let's replace it.
checkout['Language'] = checkout['Language'].fillna('ENGLISH')

# Finally, the Date column is formatted to comply with the SQL DATETIME format. UTC +10:00 indicates Brisbane Timezone, which can be ignored.
checkout['Date'] = pd.to_datetime(checkout['Date']).dt.strftime('%Y-%m-%d %H:%M:%S')

checkout.head(5)

,Title,Author,Call Number,Item Id,Item Type,Status,Language,Age,Checkout Library,Date
0,The abolition of Britain : from Winston Churchill to Theresa May,"Hitchens, Peter,",941.085 HIT,34000106965783,NONFICTION,CHECKEDOUT,English,ADULT,KEN,2024-12-09 23:57:56
1,The snakehead : an epic tale of the Chinatown underworld and the American dream,"Keefe, Patrick Radden,",364.137 KEE,34000117610220,NONFICTION,CHECKEDOUT,English,ADULT,KEN,2024-12-09 23:57:49
2,What birdo is that? : a field guide to bird-people,"Robin, Libby,",598.072 ROB,34000118196203,NONFICTION,CHECKEDOUT,English,ADULT,IPY,2024-12-09 23:55:46
3,Women making waves : trailblazing surfers in and out of the water,"Einzig, Lara,",797.32 EIN,34000116161050,NONFICTION,CHECKEDOUT,English,ADULT,IPY,2024-12-09 23:55:40
4,The next big thing,"Colley, James,",AD-PBK COL,34000119419448,AD-PBK,CHECKEDOUT,English,ADULT,IPY,2024-12-09 23:55:37


In [ ]:
# Awesome! Now save this cleaned data frame as a table before importing to MS SQL Project
checkout.to_csv('Checkout.csv', index=False)